In [3]:
# import segmentation_models_pytorch as smp
import torch
import tifffile as tiff
import os
import numpy as np

from torch import nn

In [4]:
device = 'cuda'

In [5]:
model = nn.DataParallel(torch.load('model/eff_net8.bin'))

In [6]:
model = model.eval()

In [7]:
imgs = [os.path.join('test', file) for file in os.listdir('test') if 'tiff' in file]

In [8]:
img = tiff.imread(imgs[1])

In [9]:
img.shape

(36800, 43780, 3)

In [10]:
import time

In [11]:
def inference(img_array, patch_shape=(1024, 1024), batches_num=1, device='cpu'):
    img = img_array
    img = np.transpose(img.squeeze(), (1,2,0)) if len(img.shape) == 5 else img.transpose(2, 1, 0)
    
    mask = torch.tensor(np.zeros(img.shape[1:]), device=device, dtype=torch.float)
    
    batch_patch = torch.empty(batches_num, 3, *patch_shape, device=device)
        
    for i in range(img.shape[1]//patch_shape[0]):
        for j in range(img.shape[2]//patch_shape[1]):
            slice_row, slice_col = slice(i*patch_shape[0], (i+1)*patch_shape[0]), slice(j*patch_shape[1], (j+1)*patch_shape[1])            
            
            patch = img[:, slice_row, slice_col]
            
            patch = torch.tensor(patch, device=device, dtype=torch.float)
            # (batch, channels, height, width)
            
            patch_real_shape = patch.shape
                
            batch_patch[0] = patch
                                    
            with torch.no_grad():
                output = model(batch_patch)
                
                mask[slice_row, slice_col] = output[0, 0, :patch_real_shape[1], :patch_real_shape[2]]
                torch.cuda.empty_cache()
    
    return mask